## ML con DataFrames
La interfaz de DataFrames es más moderna que la de RDDs, que en el futuro quedará obsoleta

In [1]:
##
## Se cargan librerías auxiliares que
## pueden ser de utilidad en el desarrollo
## del código
##
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
##
## Carga de las librerías de Spark
##
import findspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

findspark.init()

APP_NAME = "spark-kmeans-app"

conf = SparkConf().setAppName(APP_NAME)
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
!wget https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/snsdata.csv

--2022-11-21 18:01:24--  https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/snsdata.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2631136 (2.5M) [text/plain]
Saving to: ‘snsdata.csv’

snsdata.csv         100%[===================>]   2.51M  5.15MB/s    in 0.5s    

2022-11-21 18:01:25 (5.15 MB/s) - ‘snsdata.csv’ saved [2631136/2631136]



In [4]:
##
## Contenido del archivo
##
!head snsdata.csv

gradyear,gender,age,friends,basketball,football,soccer,softball,volleyball,swimming,cheerleading,baseball,tennis,sports,cute,sex,sexy,hot,kissed,dance,band,marching,music,rock,god,church,jesus,bible,hair,dress,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
2006,M,18.982,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,F,18.801,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,2,1,0,0,0,6,4,0,1,0,0,0,0,0,0,0,0
2006,M,18.335,69,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2006,F,18.875,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,NA,18.995,10,0,0,0,0,0,0,0,0,0,0,0,1,0,0,5,1,1,0,3,0,1,0,0,0,1,0,0,0,2,0,0,0,0,0,1,1
2006,F,,142,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
2006,F,18.93,72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0
2006,M,18.322,17,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,F,19.055

In [5]:
##
## Mueve el archivo de datos al hdfs
##
!hdfs dfs -copyFromLocal snsdata.csv /tmp/snsdata.csv

In [61]:
##
## Se carga el archivo en PySpark
##
spark_df = spark.read.load("/tmp/snsdata.csv",
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")

##
## Número de registros cargados
##
spark_df.count()

30000

In [62]:
##
## Tipos de datos de los campos del DataFrame
##
spark_df.printSchema()

root
 |-- gradyear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- friends: integer (nullable = true)
 |-- basketball: integer (nullable = true)
 |-- football: integer (nullable = true)
 |-- soccer: integer (nullable = true)
 |-- softball: integer (nullable = true)
 |-- volleyball: integer (nullable = true)
 |-- swimming: integer (nullable = true)
 |-- cheerleading: integer (nullable = true)
 |-- baseball: integer (nullable = true)
 |-- tennis: integer (nullable = true)
 |-- sports: integer (nullable = true)
 |-- cute: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- sexy: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- kissed: integer (nullable = true)
 |-- dance: integer (nullable = true)
 |-- band: integer (nullable = true)
 |-- marching: integer (nullable = true)
 |-- music: integer (nullable = true)
 |-- rock: integer (nullable = true)
 |-- god: integer (nullable = true)
 |-- church: inte

In [63]:
##
## Conteo por género
##
spark_df.groupBy('gender').count().toPandas()

,gender,count
0,F,22054
1,NA,2724
2,M,5222


In [64]:
##
## Se analizan los rangos de las variables
## para determinar si hay datos por fuera de sus
## rangos válidos. La variable `edad` contiene
## datos por fuera de la población de interés.
##
## La muestra contiene un rango de edades
## por fuera de la población de interés
##
spark_df.select('age').describe().toPandas()

,summary,age
0,count,24914
1,mean,17.993949546439772
2,stddev,7.858054477853863
3,min,3.086
4,max,106.927


In [65]:
##
## Se analizan los rangos de las variables
## para determinar si hay datos por fuera de sus
## rangos válidos. La variable `edad` contiene
## datos por fuera de la población de interés.
##
## La muestra contiene un rango de edades
## por fuera de la población de interés
##
spark_df.select('age').describe().toPandas()

,summary,age
0,count,24914
1,mean,17.993949546439772
2,stddev,7.858054477853863
3,min,3.086
4,max,106.927


In [66]:
##
## Cantidad de nulos en la columna age
##
spark_df.filter(spark_df['age'].isNull()).count()

5086

In [67]:
##
## Se seleccionan las personas entre 13 y 20 años y
## se descartan las demás observaciones
##
from pyspark.sql.functions import lit
from pyspark.sql.functions import when
from pyspark.sql.types import DoubleType

##
## Se agrega una columna con las edades entre 13 y 19,
## reemplazando por null los valores por fuera de este
## rango
##
spark_df = spark_df.withColumn(
    'age1319',
    when((spark_df['age'] >= 13) & (spark_df['age'] < 20),
         spark_df['age']
        ).otherwise(lit(None).cast(DoubleType())))

##
## Se verifican los valores en la nueva columna
##
spark_df.select('age1319').describe().toPandas()

,summary,age1319
0,count,24477
1,mean,17.25242893328433
2,stddev,1.1574649278955391
3,min,13.027
4,max,19.995


In [68]:
##
## Se calcula la edad promedio por año
## de graduación para la muestra en el
## rango de edades considerado
##
age_df = spark_df.groupby("gradyear").mean().select(['gradyear', 'avg(age1319)']).orderBy('gradyear')
age_df.show()

+--------+------------------+
|gradyear|      avg(age1319)|
+--------+------------------+
|    2006|18.655857950872626|
|    2007| 17.70617237497992|
|    2008|16.767700737100785|
|    2009|15.819573344509596|
+--------+------------------+



In [69]:
##
## Se obtienen las columnas de las
## características de intereses
##
inputCols = [a for a,_ in spark_df.dtypes]
inputCols = inputCols[3:-1]
inputCols

['friends',
 'basketball',
 'football',
 'soccer',
 'softball',
 'volleyball',
 'swimming',
 'cheerleading',
 'baseball',
 'tennis',
 'sports',
 'cute',
 'sex',
 'sexy',
 'hot',
 'kissed',
 'dance',
 'band',
 'marching',
 'music',
 'rock',
 'god',
 'church',
 'jesus',
 'bible',
 'hair',
 'dress',
 'blonde',
 'mall',
 'shopping',
 'clothes',
 'hollister',
 'abercrombie',
 'die',
 'death',
 'drunk',
 'drugs']

In [70]:
from pyspark.ml.feature import VectorAssembler

#
# Transforma una colección de features-columna en una feature-lista
# 37 es el número de elementos de cada lista. Luego vienen los valores para cada elemento
#

vectorAssembler = VectorAssembler(
    inputCols = inputCols,
    outputCol = 'rawFeatures')

spark_df = vectorAssembler.transform(spark_df)

spark_df.select('rawFeatures').show()

+--------------------+
|         rawFeatures|
+--------------------+
|(37,[0,16],[7.0,1...|
|(37,[2,11,19,20,2...|
|(37,[0,2,17,19,34...|
|(37,[11,20],[1.0,...|
|(37,[0,12,15,16,1...|
|(37,[0,12,18,19,2...|
|(37,[0,17,18,28,3...|
|(37,[0,4,12,13,19...|
|     (37,[0],[52.0])|
|(37,[0,11,14,19,2...|
|      (37,[0],[8.0])|
|(37,[0,2,22],[21....|
|(37,[0,12,19],[87...|
|          (37,[],[])|
|(37,[16,19,21,26,...|
|          (37,[],[])|
|(37,[0,20,21],[13...|
|(37,[0,3,10,19],[...|
|(37,[0,1,2,10,22]...|
|(37,[0,7,19,21,28...|
+--------------------+
only showing top 20 rows



In [71]:
spark_df.show()

+--------+------+------+-------+----------+--------+------+--------+----------+--------+------------+--------+------+------+----+---+----+---+------+-----+----+--------+-----+----+---+------+-----+-----+----+-----+------+----+--------+-------+---------+-----------+---+-----+-----+-----+-------+--------------------+
|gradyear|gender|   age|friends|basketball|football|soccer|softball|volleyball|swimming|cheerleading|baseball|tennis|sports|cute|sex|sexy|hot|kissed|dance|band|marching|music|rock|god|church|jesus|bible|hair|dress|blonde|mall|shopping|clothes|hollister|abercrombie|die|death|drunk|drugs|age1319|         rawFeatures|
+--------+------+------+-------+----------+--------+------+--------+----------+--------+------------+--------+------+------+----+---+----+---+------+-----+----+--------+-----+----+---+------+-----+-----+----+-----+------+----+--------+-------+---------+-----------+---+-----+-----+-----+-------+--------------------+
|    2006|     M|18.982|      7|         0|      

In [80]:
from pyspark.ml.feature import MinMaxScaler

##
## Se construye el modelo y se entrena
##
scalerModel = MinMaxScaler(inputCol="rawFeatures", outputCol="features").fit(spark_df)

##
## Se aplica al DataFrame original para escalar los datos
##
spark_df_scaled = scalerModel.transform(spark_df)

##
## Se imprimen los datos escalados
##
spark_df_scaled.select(['features']).show()

+--------------------+
|            features|
+--------------------+
|(37,[0,16],[0.008...|
|(37,[2,11,19,20,2...|
|(37,[0,2,17,19,34...|
|(37,[11,20],[0.05...|
|(37,[0,12,15,16,1...|
|(37,[0,12,18,19,2...|
|(37,[0,17,18,28,3...|
|(37,[0,4,12,13,19...|
|(37,[0],[0.062650...|
|(37,[0,11,14,19,2...|
|(37,[0],[0.009638...|
|(37,[0,2,22],[0.0...|
|(37,[0,12,19],[0....|
|          (37,[],[])|
|(37,[16,19,21,26,...|
|          (37,[],[])|
|(37,[0,20,21],[0....|
|(37,[0,3,10,19],[...|
|(37,[0,1,2,10,22]...|
|(37,[0,7,19,21,28...|
+--------------------+
only showing top 20 rows



In [81]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

##
## Se realiza el agrupamiento. SetK() indica la
## cantidad de grupos para los que deben obtenerse
## sus centros.
##
model = KMeans(featuresCol='features').setK(5).setSeed(1).fit(spark_df_scaled)

spark_df_scaled = model.transform(spark_df_scaled)

silhouette = ClusteringEvaluator().evaluate(spark_df_scaled)

print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.49128934366855725


In [82]:
spark_df_scaled.show()

+--------+------+------+-------+----------+--------+------+--------+----------+--------+------------+--------+------+------+----+---+----+---+------+-----+----+--------+-----+----+---+------+-----+-----+----+-----+------+----+--------+-------+---------+-----------+---+-----+-----+-----+-------+--------------------+--------------------+----------+
|gradyear|gender|   age|friends|basketball|football|soccer|softball|volleyball|swimming|cheerleading|baseball|tennis|sports|cute|sex|sexy|hot|kissed|dance|band|marching|music|rock|god|church|jesus|bible|hair|dress|blonde|mall|shopping|clothes|hollister|abercrombie|die|death|drunk|drugs|age1319|         rawFeatures|            features|prediction|
+--------+------+------+-------+----------+--------+------+--------+----------+--------+------------+--------+------+------+----+---+----+---+------+-----+----+--------+-----+----+---+------+-----+-----+----+-----+------+----+--------+-------+---------+-----------+---+-----+-----+-----+-------+-------

In [83]:
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[0.04642285 0.01810865 0.02572472 0.01256924 0.01525493 0.00739357
 0.00838003 0.0084209  0.0086072  0.00859975 0.01751248 0.0419927
 0.00411043 0.01259408 0.0126537  0.01125557 0.02924957 0.0066019
 0.00455257 0.0182379  0.02011008 0.00775115 0.01042619 0.00470974
 0.00201207 0.0295104  0.04006757 0.00082452 0.0710746  0.12505335
 0.08322155 0.02643019 0.02188129 0.01469423 0.01330203 0.02652023
 0.00930584]
[0.03361011 0.008702   0.01378927 0.00717627 0.0069497  0.00326077
 0.00334783 0.0033008  0.00584794 0.00520624 0.00959607 0.0121061
 0.0012932  0.00644962 0.00568498 0.00230983 0.01060483 0.0041546
 0.00364491 0.01009564 0.00941593 0.00535547 0.00450757 0.00349647
 0.00186909 0.00725958 0.00654461 0.00016928 0.01077153 0.01246576
 0.00528425 0.0033008  0.00269823 0.00705473 0.00702836 0.00770464
 0.00257801]
[0.04532758 0.03327114 0.02860697 0.01156256 0.04389816 0.22823383
 0.00666025 0.00760088 0.00932836 0.00688226 0.03306385 0.02100608
 0.00225844 0.00815368

In [84]:
spark_df_scaled.select('prediction').show()

+----------+
|prediction|
+----------+
|         1|
|         0|
|         1|
|         1|
|         0|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         0|
+----------+
only showing top 20 rows



In [85]:
##
## Número de patrones asignados a cada cluster
##
spark_df_scaled.groupBy('prediction').count().toPandas()

,prediction,count
0,1,23396
1,3,701
2,4,627
3,2,805
4,0,4471


In [38]:
##
## clusters a los que pertenecen los primeros patrones
##
spark_df_scaled.select(["prediction", "gender", "age", "friends"]).show()

+----------+------+------+-------+
|prediction|gender|   age|friends|
+----------+------+------+-------+
|         1|     M|18.982|      7|
|         0|     F|18.801|      0|
|         1|     M|18.335|     69|
|         1|     F|18.875|      0|
|         0|    NA|18.995|     10|
|         1|     F|  null|    142|
|         1|     F| 18.93|     72|
|         1|     M|18.322|     17|
|         1|     F|19.055|     52|
|         1|     F|18.708|     39|
|         1|     F|18.543|      8|
|         1|     F|19.463|     21|
|         1|     F|18.097|     87|
|         1|    NA|  null|      0|
|         1|     F|18.398|      0|
|         1|    NA|  null|      0|
|         1|    NA|  null|    135|
|         1|     F|18.987|     26|
|         1|     F|17.158|     27|
|         0|     F|18.497|    123|
+----------+------+------+-------+
only showing top 20 rows



In [87]:
spark_df_scaled.join(age_df, spark_df_scaled.gradyear == age_df.gradyear)
spark_df_scaled.printSchema()

root
 |-- gradyear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- friends: integer (nullable = true)
 |-- basketball: integer (nullable = true)
 |-- football: integer (nullable = true)
 |-- soccer: integer (nullable = true)
 |-- softball: integer (nullable = true)
 |-- volleyball: integer (nullable = true)
 |-- swimming: integer (nullable = true)
 |-- cheerleading: integer (nullable = true)
 |-- baseball: integer (nullable = true)
 |-- tennis: integer (nullable = true)
 |-- sports: integer (nullable = true)
 |-- cute: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- sexy: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- kissed: integer (nullable = true)
 |-- dance: integer (nullable = true)
 |-- band: integer (nullable = true)
 |-- marching: integer (nullable = true)
 |-- music: integer (nullable = true)
 |-- rock: integer (nullable = true)
 |-- god: integer (nullable = true)
 |-- church: inte

In [40]:
##
## Características demográficas de los clusters.
## Edad por cluster.
##
spark_df_scaled.groupby("prediction").mean().select(['prediction', 'avg(age1319)']).show()

+----------+------------------+
|prediction|      avg(age1319)|
+----------+------------------+
|         1| 17.30145693893734|
|         3|16.982186241610723|
|         4|16.995111324376204|
|         2| 16.98059455370652|
|         0|17.132831923890123|
+----------+------------------+



In [44]:
##
## Se agrega una columna indicadora para el genero
##
spark_df_scaled = spark_df_scaled.withColumn(
    'isFemale',
    when(spark_df_scaled['gender'] == 'F', 1).otherwise(0))

In [45]:
##
## Cantidad de mujeres por cluster.
##
spark_df_scaled.groupby("prediction").mean().select(['prediction', 'avg(isFemale)']).show()

+----------+------------------+
|prediction|     avg(isFemale)|
+----------+------------------+
|         1| 0.691143785262438|
|         3|0.9072753209700428|
|         4|0.8883572567783095|
|         2|0.8944099378881988|
|         0|0.8881681950346678|
+----------+------------------+



In [106]:
spark_df_scaled.groupby("gender").count().select('gender', 'count').show()

+------+-----+
|gender|count|
+------+-----+
|     F|22054|
|    NA| 2724|
|     M| 5222|
+------+-----+

